# 必要な準備を行う

ワークフロー実行のために必要なGINのユーザー情報を保持します。  
上から順番に実行してください。

## ◆◆◆開発メモ◆◆◆
ゆくゆくはSSOに対応し、コンテナ立ち上げ毎のユーザー名、パスワード、メールアドレスの入力を避けたい

## 1.ユーザー情報を情報を入力する

以下のセルを実行してデータガバナンス機能のユーザー名、パスワード、メールアドレスを入力してください。

In [ ]:
from IPython.display import clear_output
import getpass
name = input("ユーザー名：")
password = getpass.getpass("パスワード：")
email = input("メールアドレス：")
clear_output()

## 2.ユーザー情報を登録する

### .netrcにユーザー情報を追加する

In [ ]:
!echo -e "machine dg02.dg.rcos.nii.ac.jp\nlogin $name\npassword $password" > ~/.netrc

## 3. 高性能実験環境利用のために必要な設定をする

高性能実験環境を利用している場合には、以降のセルを実行してください。

### 3.1. アカウント情報の入力

以下のセルを実行し、表示されるフォームにアカウント情報を入力してください。

In [ ]:
from IPython.display import clear_output
import getpass
name_mdx = input("高性能実験環境におけるSSHユーザ名：")
clear_output()

### 3.2. SSH秘密鍵の設置

左上のバナーを押下し、ファイル一覧画面に遷移してください。  
遷移後、id_rsaファイルをドラッグアンドドロップによりアップロードしてください。  
アップロード後、以下のセルを実行してください。

In [ ]:
!mkdir -p /home/jovyan/.ssh/
!mv ~/id_rsa ~/.ssh/
!chmod 600 ~/.ssh/id_rsa

with open("/home/jovyan/.ssh/config", "a") as f:
     f.write("Host mdx\n")
     f.write("  Hostname 163.220.176.51\n")
     f.write("  User " + name_mdx + "\n")
     f.write("  Port 22\n")
     f.write("  IdentityFile ~/.ssh/id_rsa\n")
     f.write("  StrictHostKeyChecking no\n")

### 3.3. 実験活動のためのディレクトリ初期化

In [ ]:
import os
tmp = os.environ["BINDER_REF_URL"]
tmp = tmp.split("/")
tmp = tmp[len(tmp) - 1]
repo_name = tmp[:tmp.find(".")]

cmd = "mkdir ./" + repo_name + "/"
!ssh mdx $cmd

!scp -r ~/input_data/ mdx:~/$repo_name/input_data
!scp -r ~/output_data/ mdx:~/$repo_name/output_data
!scp -r ~/source/ mdx:~/$repo_name/source